## Quora Insincere Questions Classification
#### Aleix Casellas Comas, Rubén Barco Terrones, Andreu Masdeu Ninot, Pablo Lázaro Terrones, Marco Gani Remane

### Libraries

In [1]:
import pandas as pd
import numpy as np
import sklearn
import re
import multiprocessing
import gensim.models.word2vec as w2v
import os

from sklearn import model_selection

C:\Users\ruben\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


### ETL
#### Split data into train and test

In [2]:
dir_data = 'C:/Users/ruben/Documents/Máster Data Science/2º Cuatrimestre/Natural Languaje Processing/ML_for_NLP-master/project_1/quora/'
train_data = pd.read_csv(dir_data+'train.csv')
train_data = train_data.drop(train_data.index[420816]) #'"'
train_data = train_data.reset_index(drop=True)
train_data = train_data.drop(train_data.index[792938]) #'Do '
train_data = train_data.reset_index(drop=True)
train_data = train_data.drop(train_data.index[995255]) #'W'
train_data = train_data.reset_index(drop=True)

train_data

,qid,question_text,target
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0
5,00004f9a462a357c33be,"Is Gaza slowly becoming Auschwitz, Dachau or T...",0
6,00005059a06ee19e11ad,Why does Quora automatically ban conservative ...,0
7,0000559f875832745e2e,Is it crazy if I wash or wipe my groceries off...,0
8,00005bd3426b2d0c8305,"Is there such a thing as dressing moderately, ...",0
9,00006e6928c5df60eacb,Is it just me or have you ever been in this ph...,0


In [3]:
X_train, X_test = model_selection.train_test_split(train_data, test_size=0.2, stratify=train_data['target'], random_state=123)

In [4]:
X_train.shape, X_test.shape

((1044895, 3), (261224, 3))

In [5]:
y_train =  X_train['target'].values
y_train.shape

(1044895,)

In [6]:
y_test = X_test['target'].values
y_test.shape

(261224,)

In [7]:
x_train = X_train['question_text'].values
x_train, len(x_train)

(array([ 'How is the writing style and structure in the novel "Germinal" by Émile Zola depicted?',
        'Is a debtor, a bonded labor? Why',
        'What are the best ways to develop leads?', ...,
        'Is the discount rate of buying one share of a stock equal to the discount rate of buying ten shares?',
        'What is the best way to get a personal loan in Kenya?',
        'Do you think a piloted airplane could fly under the Deception Pass Bridge?'], dtype=object),
 1044895)

In [8]:
x_test = X_test['question_text'].values
x_test, len(x_test)

(array(['What is the minimum salary required for American Express Card?',
        'Can you make French fries only out of russet potatoes?',
        'How is the mark vs relative grade at NITC? What would be the pass mark for maths 1 usually? No one has answered this type of question on Quora . How much marks required for each grade?',
        ...,
        'What is the maximum size Transmission/Front sprocket that can be used for a Bajaj Avenger 220?',
        'What should I do if I have a small penis at 15-years-old?',
        'In which direction does spiders make its web?'], dtype=object),
 261224)

In [9]:
def sentence_to_wordlist(raw):
    clean = raw#    clean = re.sub("[^a-zA-Z]"," ", raw)
    clean = clean.lower()
    words = clean.split()
    return words

In [10]:
sentences = []
for raw_sentence in x_train:
    if len(raw_sentence) > 0:
        sentences.append(sentence_to_wordlist(raw_sentence))
for raw_sentence in x_test:
    if len(raw_sentence) > 0:
        sentences.append(sentence_to_wordlist(raw_sentence))

In [11]:
len(sentences), len(x_train)+len(x_test)

(1306119, 1306119)

In [12]:
sentences[0]

['how',
 'is',
 'the',
 'writing',
 'style',
 'and',
 'structure',
 'in',
 'the',
 'novel',
 '"germinal"',
 'by',
 'émile',
 'zola',
 'depicted?']

In [13]:
token_count = sum([len(sentence) for sentence in sentences])
print("The book corpus contains {0:,} tokens".format(token_count))

The book corpus contains 16,723,073 tokens


## Word2vec - 10 epochs / 300 features
### Train word2vec

In [14]:
num_features = 300
num_epochs   = 10

# Minimum word count threshold.
min_word_count = 0

# Number of threads to run in parallel.
num_workers = multiprocessing.cpu_count()

# Context window length.
context_size = 3

# Downsample setting for frequent words.
#0 - 1e-5 is good for this
downsampling = 1e-3

seed = 1

#optional Training algorithm: 1 for skip-gram; otherwise CBOW
sg = 1

In [15]:
word2vec = w2v.Word2Vec(
    sg=sg,
    seed=seed,
    workers=num_workers,
    size=num_features,
    min_count=min_word_count,
    window=context_size,
    sample=downsampling)

In [16]:
word2vec.build_vocab(sentences, keep_raw_vocab=True)

In [17]:
word2vec.corpus_count

1306119

In [18]:
len(word2vec.vocabulary.raw_vocab)

450473

In [19]:
total_examples = len(sentences)

In [20]:
%%time
word2vec.train(sentences,
               epochs = num_epochs,
               total_examples=total_examples)

Wall time: 4min 32s


(124297292, 167230730)

In [22]:
foldername = "C:/Users/ruben/Documents/Máster Data Science/2º Cuatrimestre/Natural Languaje Processing/ML_for_NLP-master/project_1/quora/saved_models/" + "w2v_" + str(num_features) +"features_"+ str(context_size) +"contextsize_"+str(num_epochs)+"epochs"
modelname  = "word2vec_" + str(num_features) +"features"+ str(context_size) +"contextsize"+ str(num_epochs)+"epochs.w2v"

if not os.path.exists(foldername):
    os.makedirs(foldername)
    word2vec.save(os.path.join(foldername, modelname))
else:
    print("folder {} already exists".format(foldername))

### Predicting from a Word2vec averaged representation

In [23]:
def doc_to_vec(sentence, word2vec):
    word_list    = sentence_to_wordlist(sentence)
    word_vectors = []
    for w in word_list:
        word_vectors.append(word2vec.wv.get_vector(w))

    return np.mean(word_vectors,axis=0)

In [24]:
X_tr = np.zeros((len(x_train), num_features))
y_tr = y_train
n_samples = X_tr.shape[0]

for i in range(n_samples):
    X_tr[i,:] = doc_to_vec(x_train[i], word2vec)

In [25]:
# X_tr[0,:]

In [26]:
X_te = np.zeros((len(x_test), num_features))
y_te = y_test
n_samples = X_te.shape[0]
for i in range(n_samples):
    X_te[i,:] = doc_to_vec(x_test[i], word2vec)

In [27]:
X_tr.shape, X_te.shape

((1044895, 300), (261224, 300))

### Predicting with a Logistic Regression 

In [28]:
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression
from sklearn.metrics import accuracy_score, f1_score
logreg = LogisticRegression(n_jobs=1, C=1e5)
logreg.fit(X_tr, y_tr)

LogisticRegression(C=100000.0, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [29]:
y_pred = logreg.predict(X_tr)
print('Train accuracy: {}'.format(accuracy_score(y_tr, y_pred)))
print('Train F1 score: {}'.format(f1_score(y_tr, y_pred)))

y_pred = logreg.predict(X_te)
print('Test accuracy: {}'.format(accuracy_score(y_te, y_pred)))
print('Test F1 score: {}'.format(f1_score(y_te, y_pred)))

Train accuracy: 0.9466252589973155
Train F1 score: 0.4246794375844603
Test accuracy: 0.9465056809481518
Test F1 score: 0.42394261686866186


## Word2vec - 20 epochs / 300 features
### Train word2vec

In [30]:
num_features = 300
num_epochs   = 20

# Minimum word count threshold.
min_word_count = 0

# Number of threads to run in parallel.
num_workers = multiprocessing.cpu_count()

# Context window length.
context_size = 7

# Downsample setting for frequent words.
#0 - 1e-5 is good for this
downsampling = 1e-3

seed = 1

#optional Training algorithm: 1 for skip-gram; otherwise CBOW
sg = 1

In [31]:
word2vec = w2v.Word2Vec(
    sg=sg,
    seed=seed,
    workers=num_workers,
    size=num_features,
    min_count=min_word_count,
    window=context_size,
    sample=downsampling)

In [32]:
word2vec.build_vocab(sentences, keep_raw_vocab=True)

In [33]:
word2vec.corpus_count

1306119

In [34]:
len(word2vec.vocabulary.raw_vocab)

450473

In [35]:
total_examples = len(sentences)

In [36]:
%%time
word2vec.train(sentences,
               epochs = num_epochs,
               total_examples=total_examples)

Wall time: 15min 10s


(248592853, 334461460)

In [37]:
foldername = "C:/Users/ruben/Documents/Máster Data Science/2º Cuatrimestre/Natural Languaje Processing/ML_for_NLP-master/project_1/quora/saved_models/" + "w2v_" + str(num_features) +"features_"+ str(context_size) +"contextsize_"+str(num_epochs)+"epochs"
modelname  = "word2vec_" + str(num_features) +"features"+ str(context_size) +"contextsize"+ str(num_epochs)+"epochs.w2v"

if not os.path.exists(foldername):
    os.makedirs(foldername)
    word2vec.save(os.path.join(foldername, modelname))
else:
    print("folder {} already exists".format(foldername))

### Predicting from a Word2vec averaged representation

In [38]:
def doc_to_vec(sentence, word2vec):
    word_list    = sentence_to_wordlist(sentence)
    word_vectors = []
    for w in word_list:
        word_vectors.append(word2vec.wv.get_vector(w))

    return np.mean(word_vectors,axis=0)

In [39]:
X_tr = np.zeros((len(x_train), num_features))
y_tr = y_train
n_samples = X_tr.shape[0]

for i in range(n_samples):
    X_tr[i,:] = doc_to_vec(x_train[i], word2vec)

In [40]:
X_te = np.zeros((len(x_test), num_features))
y_te = y_test
n_samples = X_te.shape[0]
for i in range(n_samples):
    X_te[i,:] = doc_to_vec(x_test[i], word2vec)

In [41]:
X_tr.shape, X_te.shape

((1044895, 300), (261224, 300))

### Predicting with a Logistic Regression 

In [42]:
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression
from sklearn.metrics import accuracy_score, f1_score
logreg = LogisticRegression(n_jobs=1, C=1e5)
logreg.fit(X_tr, y_tr)

LogisticRegression(C=100000.0, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [43]:
y_pred = logreg.predict(X_tr)
print('Train accuracy: {}'.format(accuracy_score(y_tr, y_pred)))
print('Train F1 score: {}'.format(f1_score(y_tr, y_pred)))

y_pred = logreg.predict(X_te)
print('Test accuracy: {}'.format(accuracy_score(y_te, y_pred)))
print('Test F1 score: {}'.format(f1_score(y_te, y_pred)))

Train accuracy: 0.9472128778489705
Train F1 score: 0.4375121100561907
Test accuracy: 0.9468884941659266
Test F1 score: 0.43546549479166663
